# DATA20001 Deep Learning - Group Project
## Image project

**Due Wednesday December 13, before 23:59.**

The task is to learn to assign the correct labels to a set of images.  The images are originally from a photo-sharing site and released under Creative Commons-licenses allowing sharing.  The training set contains 20 000 images. We have resized them and cropped them to 128x128 to make the task a bit more manageable.

We're only giving you the code for downloading the data. The rest you'll have to do yourselves.

Some comments and hints particular to the image project:

- One image may belong to many classes in this problem, i.e., it's a multi-label classification problem. In fact there are images that don't belong to any of our classes, and you should also be able to handle these correctly. Pay careful attention to how you design the outputs of the network (e.g., what activation to use) and what loss function should be used.

- As the dataset is pretty imbalanced, don't focus too strictly on the outputs being probabilistic. (Meaning that the right threshold for selecting the label might not be 0.5.)

- Image files can be loaded as numpy matrices for example using `imageio.imread`. Most images are color, but a few grayscale. You need to handle the grayscale ones somehow as they would have a different number of color channels (depth) than the color ones.

- Loading all the images into one big matrix as we have done in the exercises is not feasible (e.g. the virtual servers in CSC have only 3 GB of RAM). You need to load the images in smaller chunks for the training. This shouldn't be a problem we are doing mini-batch training anyway, and thus we don't need to keep all the images in memory. You can simply pass you current chunk of images to `model.fit()` as it remembers the weights from the previous run.

- You need to think carefully about how you load the annotations and match them up with the corresponding images, especially as you are loading them in smaller chunks.

## Download the data

In [ ]:
from keras.utils.data_utils import get_file

database_path = 'train/'

dl_file='dl2017-image-proj.zip'
dl_url='https://www.cs.helsinki.fi/u/mvsjober/misc/'
get_file(dl_file, dl_url+dl_file, cache_dir='./', cache_subdir=database_path, extract=True)

The above command downloaded and extracted the data files into the `train` subdirectory.

The images can be found in `train/images`, and are named as `im1.jpg`, `im2.jpg` and so on until `im20000.jpg`.

The class labels, or annotations, can be found in `train/annotations` as `CLASSNAME.txt`, where CLASSNAME is one of the fourteen classes: *baby, bird, car, clouds, dog, female, flower, male, night, people, portrait, river, sea,* and *tree*.

Each annotation file is a simple text file that lists the images that depict that class, one per line. The images are listed with their number, not the full filename. For example `5969` refers to the image `im5969.jpg`.

## Your stuff goes here ...

### Import

In [ ]:
%matplotlib inline

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.layers.convolutional import *
from keras.preprocessing.image import *
from keras import backend as K
from keras.utils.vis_utils import model_to_dot
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from scipy import misc
from sklearn.metrics import f1_score
from IPython.display import SVG

### Pre-process

In [ ]:
class_indices = {0:"baby",
                 1:"bird",
                 2:"car",
                 3:"clouds",
                 4:"dog",
                 5:"female",
                 6:"flower",
                 7:"male",
                 8:"night",
                 9:"people",
                 10:"portrait",
                 11:"river",
                 12:"sea",
                 13:"tree"}

#### Label and file name loading

In [ ]:
N = 20000
num_class = len(class_indices)


img_list = [database_path+"images/"+"im"+str(x)+".jpg" for x in range(1,N+1)]
labels = np.zeros((N,num_class))

for k,v in class_indices.items():
    with open(database_path+"annotations/"+v+".txt","r") as fp:
        for line in fp:
            labels[int(line)-1,k] = 1

#### Spliting train set and validation set

In [ ]:
train_val_ratio = 0.8 #train:vad = 0.8
img_size = (32,32)

x_train_list, x_val_list, y_train, y_val = train_test_split(img_list,labels,test_size = 1-train_val_ratio,shuffle=True)
image_load_func = lambda path: misc.imresize(misc.imread(path,mode="RGB"),size=img_size)

In [ ]:
img0 = image_load_func(x_train_list[0])
for l in range(3):
    plt.subplot(1, 4, l+1)
    plt.imshow(img0[:,:,l])
    plt.axis('off')
plt.subplot(1, 4, 4)
plt.imshow(img0)
plt.axis('off')

if sum(y_train[0]==1)==0:
    print("No label")
else:
    for i in np.argwhere(y_train[0]==1).flatten():
        print(class_indices[i])

In [ ]:
x_train = np.stack([image_load_func(file) for file in x_train_list])
x_val = np.stack([image_load_func(file) for file in x_val_list])
print(x_train.shape,x_val.shape)

### Model Building

In [ ]:
from densenet import *

#### Standard DenseNet with output layer adjusted

In [ ]:
nb_filter=32
weight_decay=0.00001
dropout_rate=0.2
nb_dense_block=3
nb_layers=2
growth_rate=12

model = DenseNet(num_class, x_train.shape[1:], nb_layers*3+4, nb_dense_block, growth_rate,
             nb_filter, dropout_rate=None, weight_decay=1E-4)
model.compile(loss='categorical_crossentropy', 
              optimizer='adam')

print(model.summary())

In [ ]:
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
%%time

epochs = 3

history = model.fit(x_train, 
                    y_train, 
                    epochs=epochs, 
                    batch_size=64,
                    verbose=1)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['loss'])
plt.title('loss')

In [ ]:
y_pred = np.rint(model.predict(x_val))
print("micro averaged f1 score:", f1_score(y_val, y_pred, average='micro'))

#### Customized DenseNet

In [ ]:
nb_filter=32
weight_decay=0.00001
dropout_rate=0.2
nb_dense_block=3
nb_layers=2
growth_rate=1


model_input = Input(shape=x_train.shape[1:])

x = Conv2D(nb_filter, (3, 3),
               kernel_initializer="he_uniform",
               padding="same",
               name="initial_conv2D",
               use_bias=True,
               kernel_regularizer=l2(weight_decay))(model_input)
#blocks
for block_idx in range(nb_dense_block - 1):
    x, nb_filter = denseblock(x, nb_layers, nb_filter, growth_rate,
                              dropout_rate=dropout_rate)
    # add transition
    x = transition(x, nb_filter, dropout_rate=dropout_rate,
                   weight_decay=weight_decay)

#last block 
x, nb_filter = denseblock(x, nb_layers, nb_filter, growth_rate,
                              dropout_rate=dropout_rate,
                              weight_decay=weight_decay)

x = BatchNormalization(axis=1,
                       gamma_regularizer=l2(weight_decay),
                       beta_regularizer=l2(weight_decay))(x)
x = Activation('relu')(x)
x = GlobalAveragePooling2D(data_format=K.image_data_format())(x)
#linear

model_output = Dense(num_class, activation='sigmoid')(x)

model = Model(inputs=[model_input], outputs=[model_output], name="DenseNet")

model.compile(loss='categorical_crossentropy', 
              optimizer='adam')

print(model.summary())

In [ ]:
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [ ]:
%%time

epochs = 3

history = model.fit(x_train, 
                    y_train, 
                    epochs=epochs, 
                    batch_size=64,
                    verbose=1)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['loss'])
plt.title('loss')

In [ ]:
y_pred = np.rint(model.predict(x_val))
print("micro averaged f1 score:", f1_score(y_val, y_pred, average='micro'))

#### Classic ConvNet

In [ ]:
model=Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=x_train.shape[1:],activation="relu"))

model.add(Flatten())
model.add(Dense(num_class,activation="sigmoid"))


model.compile(loss='categorical_crossentropy', 
              optimizer=opt)
print(model.summary())

In [ ]:
%%time

epochs = 3

history = model.fit(x_train, 
                    y_train, 
                    epochs=epochs, 
                    batch_size=64,
                    verbose=1)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['loss'])
plt.title('loss')

In [ ]:
y_pred = np.rint(model.predict(x_val))
print("micro averaged f1 score:", f1_score(y_val, y_pred, average='micro'))

## Save your model

It might be useful to save your model if you want to continue your work later, or use it for inference later.

In [ ]:
model.save('model.h5')

The model file should now be visible in the "Home" screen of the jupyter notebooks interface.  There you should be able to select it and press "download".

## Predict for test set

You will be asked to return your prediction for the testset.  These should be returned as a matrix with one row for each test set image.  Each row contains a binary prediction for each label, 1 if it's present in the image, and 0 if not. The order of the labels is as follows (alphabetic order of the label names):

    baby bird car clouds dog female flower male night people portrait river sea tree

An example row could like like this if your system predicts the presense of a bird and clouds:

    0 1 0 1 0 0 0 0 0 0 0 0 0 0
    
If you have the matrix prepared in `y` (e.g., by calling `y=model.predict(x_test)`) you can use the following function to save it to a text file.

In [ ]:
np.savetxt('results.txt', y, fmt='%d')